<a href="https://colab.research.google.com/github/aiswarya-180/Bank-Customer-Churn-Prediction/blob/main/MRP_Baseline_Model_(All_Features%2C_No_Sampling).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Purpose: Understand raw model performance**



In [ ]:
!pip install catboost


In [ ]:
# mount google
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# imports
import pandas as pd
import numpy as np
import math

# Visualization
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, roc_auc_score

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [ ]:
df_1 = pd.read_csv('/content/drive/My Drive/MRP/after_EDA.csv')
df_1.head()

,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,0,45,1,3,High School,Married,$60K - $80K,Blue,39,5,1,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061
1,0,49,0,5,Graduate,Single,Less than $40K,Blue,44,6,1,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105
2,0,51,1,3,Graduate,Married,$80K - $120K,Blue,36,4,1,0,3418.0,0,3418.0,2.594,1887,20,2.333,0.000
3,0,40,0,4,High School,Unknown,Less than $40K,Blue,34,3,4,1,3313.0,2517,796.0,1.405,1171,20,2.333,0.760
4,0,40,1,3,Uneducated,Married,$60K - $80K,Blue,21,5,1,0,4716.0,0,4716.0,2.175,816,28,2.500,0.000


In [ ]:
df_1.shape

(10127, 20)

In [ ]:
categorical_cols = df_1.select_dtypes(include=['object']).columns
categorical_cols

Index(['Education_Level', 'Marital_Status', 'Income_Category',
       'Card_Category'],
      dtype='object')

# Catboost without encoding

In [ ]:
X = df_1.drop("Attrition_Flag", axis=1)
y = df_1["Attrition_Flag"]


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [ ]:
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report

# Initialize CatBoost
cat_model = CatBoostClassifier(verbose=0, random_state=42)

# Train with raw categorical features
cat_model.fit(X_train, y_train, cat_features=categorical_cols.tolist())
# X_train includes all features (numerical + categorical).The cat_features parameter tells CatBoost which columns in X_train are categorical — so it can apply specialized handling

# Predict and evaluate
y_pred_cat = cat_model.predict(X_test)
print("CatBoost (native categorical support):\n", classification_report(y_test, y_pred_cat))


CatBoost (native categorical support):
               precision    recall  f1-score   support

           0       0.98      0.99      0.98      1701
           1       0.96      0.87      0.92       325

    accuracy                           0.97      2026
   macro avg       0.97      0.93      0.95      2026
weighted avg       0.97      0.97      0.97      2026



# One Hot Encoding

In [ ]:
# Apply one-hot encoding
df_encoded = pd.get_dummies(df_1, columns=categorical_cols, drop_first=True)

df_encoded.head()

,Attrition_Flag,Customer_Age,Gender,Dependent_count,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,...,Marital_Status_Single,Marital_Status_Unknown,Income_Category_$40K - $60K,Income_Category_$60K - $80K,Income_Category_$80K - $120K,Income_Category_Less than $40K,Income_Category_Unknown,Card_Category_Gold,Card_Category_Platinum,Card_Category_Silver
0,0,45,1,3,39,5,1,3,12691.0,777,...,False,False,False,True,False,False,False,False,False,False
1,0,49,0,5,44,6,1,2,8256.0,864,...,True,False,False,False,False,True,False,False,False,False
2,0,51,1,3,36,4,1,0,3418.0,0,...,False,False,False,False,True,False,False,False,False,False
3,0,40,0,4,34,3,4,1,3313.0,2517,...,False,True,False,False,False,True,False,False,False,False
4,0,40,1,3,21,5,1,0,4716.0,0,...,False,False,False,True,False,False,False,False,False,False


# Seperate X & Y

In [ ]:
X_encoded = df_encoded.drop(columns=['Attrition_Flag'])
y_encoded = df_encoded['Attrition_Flag']


In [ ]:
X_train_enc, X_test_enc, y_train_enc, y_test_enc = train_test_split(
    X_encoded, y_encoded, stratify=y_encoded, test_size=0.2, random_state=42
)


In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_enc)
X_test_scaled = scaler.transform(X_test_enc)

# Traditional ML Models

In [ ]:


# Logistic Regression
lr = LogisticRegression(random_state=42)
lr.fit(X_train_scaled, y_train_enc)
print("Logistic Regression:\n", classification_report(y_test_enc, lr.predict(X_test_scaled)))

# SVM
svm = SVC(probability=True, random_state=42)
svm.fit(X_train_scaled, y_train_enc)
print("SVM:\n", classification_report(y_test_enc, svm.predict(X_test_scaled)))

# Naive Bayes
nb = GaussianNB()
nb.fit(X_train_scaled, y_train_enc)
print("Naive Bayes:\n", classification_report(y_test_enc, nb.predict(X_test_scaled)))


Logistic Regression:
               precision    recall  f1-score   support

           0       0.92      0.97      0.94      1701
           1       0.77      0.54      0.63       325

    accuracy                           0.90      2026
   macro avg       0.84      0.75      0.79      2026
weighted avg       0.89      0.90      0.89      2026

SVM:
               precision    recall  f1-score   support

           0       0.92      0.98      0.95      1701
           1       0.87      0.56      0.68       325

    accuracy                           0.92      2026
   macro avg       0.90      0.77      0.82      2026
weighted avg       0.91      0.92      0.91      2026

Naive Bayes:
               precision    recall  f1-score   support

           0       0.92      0.93      0.92      1701
           1       0.60      0.57      0.58       325

    accuracy                           0.87      2026
   macro avg       0.76      0.75      0.75      2026
weighted avg       0.87      0.8

# Ensemble Models

In [ ]:
# Random Forest
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train_enc, y_train_enc)
print("Random Forest:\n", classification_report(y_test_enc, rf.predict(X_test_enc)))


# Gradient Boosting
gb = GradientBoostingClassifier(random_state=42)
gb.fit(X_train_enc, y_train_enc)
print("Gradient Boosting:\n", classification_report(y_test_enc, gb.predict(X_test_enc)))


# XGBoost
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_model.fit(X_train_enc, y_train_enc)
print("XGBoost:\n", classification_report(y_test_enc, xgb_model.predict(X_test_enc)))


# LightGBM
lgb_model = lgb.LGBMClassifier(random_state=42)
lgb_model.fit(X_train_enc, y_train_enc)
print("LightGBM:\n", classification_report(y_test_enc, lgb_model.predict(X_test_enc)))



Random Forest:
               precision    recall  f1-score   support

           0       0.96      0.99      0.97      1701
           1       0.94      0.77      0.85       325

    accuracy                           0.96      2026
   macro avg       0.95      0.88      0.91      2026
weighted avg       0.96      0.96      0.95      2026

Gradient Boosting:
               precision    recall  f1-score   support

           0       0.96      0.99      0.98      1701
           1       0.96      0.81      0.88       325

    accuracy                           0.96      2026
   macro avg       0.96      0.90      0.93      2026
weighted avg       0.96      0.96      0.96      2026



/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [21:42:40] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost:
               precision    recall  f1-score   support

           0       0.98      0.99      0.98      1701
           1       0.93      0.87      0.90       325

    accuracy                           0.97      2026
   macro avg       0.95      0.93      0.94      2026
weighted avg       0.97      0.97      0.97      2026

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 1302, number of negative: 6799
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000565 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2058
[LightGBM] [Info] Number of data points in the train set: 8101, number of used features: 31
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.160721 -> initscore=-1.652874
[LightGBM] [Info] Start training from score -1.652874
LightGBM:
               pre

#  Deep Learning Model

In [ ]:

# Define model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC()])

# Train model
model.fit(X_train_scaled, y_train, validation_data=(X_test_scaled, y_test), epochs=30, batch_size=32, verbose=1)

# Evaluate
y_pred_dl = (model.predict(X_test_scaled) > 0.5).astype("int32")
print("Deep Learning:\n", classification_report(y_test, y_pred_dl))


Epoch 1/30


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


254/254 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.7602 - auc_1: 0.6056 - loss: 0.5105 - val_accuracy: 0.8875 - val_auc_1: 0.8635 - val_loss: 0.2942
Epoch 2/30
254/254 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8800 - auc_1: 0.8641 - loss: 0.3057 - val_accuracy: 0.8944 - val_auc_1: 0.8896 - val_loss: 0.2736
Epoch 3/30
254/254 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8963 - auc_1: 0.8964 - loss: 0.2655 - val_accuracy: 0.8963 - val_auc_1: 0.9039 - val_loss: 0.2572
Epoch 4/30
254/254 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9007 - auc_1: 0.9049 - loss: 0.2625 - val_accuracy: 0.8968 - val_auc_1: 0.9165 - val_loss: 0.2486
Epoch 5/30
254/254 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9061 - auc_1: 0.9249 - loss: 0.2341 - val_accuracy: 0.9023 - val_auc_1: 0.9243 - val_loss: 0.2354
Epoch 6/30
254/254 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9136 - auc_1: 0.9309 - loss: 0.2240 - val_accuracy: 0.9028 - val_auc_1: 0.9302 - val_loss: 0.2274
Epoch 7/30
254/254 ━━━━━━━━